In [236]:
# IMPORT VITAL LABRARIES
import pandas as pd
from datetime import date
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from random import randint
from bs4 import BeautifulSoup
import xlsxwriter
import re

print('- Finish importing package')

- Finish importing package


In [59]:
# IMPORT USER ACCOUNT
## Create .txt file containing login data

credential = open('loginCRE_JOBS.txt') ### Open the txt file
line = credential.readlines () ### Read the file by line
username = line [0] ### Line 0 = username
password = line [1]
keyword = line [2]
location = line [3]
print('- Finish importing login information')

- Finish importing login information


In [60]:
# TASK 1: LOGIN TO LINKEDIN WEBSITE

## Open Chrome and Login the site
driver = webdriver.Chrome()
link = 'https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin'   
driver.get(link)
print('- Finish initializing driver')

## Expand browser size
driver.maximize_window()

## Create action chain object
action = ActionChains(driver)

## Waiting 3 - 6s until an element is successfully loaded 
sleep(randint(3,6))

## Locate the box and insert username
email_box = driver.find_element(By.ID, 'username')
email_box.send_keys(username)
sleep(randint(3,6))
print('- Finish importing email')

## Locate the box and insert password
password_box = driver.find_element(By.NAME, 'session_password')
password_box.send_keys(password)
sleep(randint(3,6))
print('- Finish importing password')


- Finish initializing driver
- Finish importing email
- Finish importing password


In [61]:
# TASK 2: SEARCH FOR THE JOB WE WANT TO CRAWL

## Locate the search box
search_box = driver.find_element(By.CLASS_NAME, 'search-global-typeahead__input')
sleep(randint(3,6))
print('- Finish locate search box')

## Insert the searching key (Data Analyst)
search_box.send_keys(keyword)
search_box.send_keys(Keys.RETURN)
sleep(randint(9,10))
print('- Finish insert keyword')

## Locate the job catagory
job_cata = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div[2]/div/div[1]/div/div/div/section/ul/li[1]/button')
job_cata.click()
sleep(randint(3,6))
print('- Finish select job catagory')

## See all of the job result
all_job = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div[2]/div/div[1]/main/div/div/div[1]/div/div[2]/a')
all_job.click()
sleep(randint(3,6))
print('- Finish see all results')


- Finish locate search box
- Finish insert keyword
- Finish select job catagory
- Finish see all results


In [62]:
## Locate the location search box
location_search_box = driver.find_element(By.XPATH, '/html/body/div[5]/header/div/div/div/div[2]/div[2]/div/div/input[1]')
location_search_box.click()
sleep(randint(3,6))
print('- Finish locate location search box')

## Command+A and Delete to clean the default location
location_search_box.send_keys(Keys.COMMAND + "a")
location_search_box.send_keys(Keys.DELETE)
sleep(randint(3,6))
print('- Finish reset location search box')

## Send the searching location
location_search_box.send_keys(location)
location_search_box.send_keys(Keys.RETURN)
location_search_box.send_keys(Keys.ENTER)
location_search_button = driver.find_element(By.XPATH, '/html/body/div[5]/header/div/div/div/div[2]/button[1]').click()
sleep(randint(3,6))
print('- Finish locate search box')


- Finish locate location search box
- Finish reset location search box
- Finish locate search box


In [63]:
# TASK 3: STARTING COLLECTING JOBS 

## Create empty target lists
title = []
skill = []
company = []
job_type = []
workplace_type = []
company_size = []
date_posted = []
location = []
total_applicants = []
link = []
i = 1

## Enter the number of page that you want to scrape
input_page = int (input ( 'How many pages you want to scrape: '))  


while i <= input_page:
    ### Be careful with the word 'f' before the css selector, without 'f' the variable i will be useless
    driver.find_element(By.CSS_SELECTOR, f'button[aria-label = "Page {i}"]').click() 
    sleep(1)
    
    ### Because the current page has 2 scroll downs, we must specify which will be scrolled
    jobs_block = driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div[4]/div/div/main/div/div[1]')
    jobs_list= jobs_block.find_elements(By.CSS_SELECTOR, '.jobs-search-results__list-item')
    for job in jobs_list:
        
        ## Scroll to the bottom of the page and pull up at the job list block to load the page before find_elemennts
        driver.execute_script("scrollBy(0,1000);")
        sleep(1)
        driver.execute_script("arguments[0].scrollIntoView();", job)
        driver.execute_script("scrollBy(0,-1000);")
        driver.execute_script("arguments[0].scrollIntoView();", job)
        job.find_element(By.CSS_SELECTOR, '.job-card-container--clickable').click()
        sleep(1)

        ## Find and collect the DATE POSTED element
        try:
            driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div[4]/div/div/main/div/div[2]/div').click()
            date_posted.append(driver.find_element(By.CSS_SELECTOR, '.jobs-unified-top-card__posted-date').text)
        except:
            date_posted.append('Unknown')
        sleep(1)

        ## Find and collect the TITLE element
        try:
            title.append(job.find_element(By.CSS_SELECTOR, '.job-card-list__title').text)
        except:
            title.append('Unknown')

        ## Find and collect the SKILL element
        ### Because the SKILL is hidden below, you have to pull down in the job description block
        try:
            driver.find_element(By.XPATH, '/html/body/div[5]/div[3]/div[4]/div/div/main/div/div[2]/div').click()
            driver.find_element(By.CSS_SELECTOR, 'a[href = "#HYM"]').click()
            skill.append(driver.find_element(By.CLASS_NAME, 'job-details-how-you-match__skills-item-subtitle').text)
        except:
            skill.append('No mention')
        
        ## Find and collect the COMPANY element
        try:
            company.append(job.find_element(By.CSS_SELECTOR, '.job-card-container__primary-description').text)
        except:
            company.append('Unknown')
        
         ## Find and collect the JOB TYPE element       
        try:
            job_type.append(driver.find_element(By.CSS_SELECTOR, 'li[class = "jobs-unified-top-card__job-insight"]:nth-child(1) > span').text)
        except:
            job_type.append('Unknown')
            
         ## Find and collect the WORKPLACE TYPE element           
        try:
            workplace_type.append(driver.find_element(By.CSS_SELECTOR, '.jobs-unified-top-card__workplace-type').text)
        except:
            workplace_type.append('Unknown')
        
        ## Find and collect the LOCATION element
        try:
            location.append(driver.find_element(By.CSS_SELECTOR, 'span[class = "jobs-unified-top-card__bullet"]').text)
        except:
            location.append('Not mention')
            
         ## Find and collect the COMPANY SIZE element           
        try:
            company_size.append(driver.find_element(By.CSS_SELECTOR, 'li[class = "jobs-unified-top-card__job-insight"]:nth-child(2) > span').text)
        except:
            company_size.append('No mention')

        ## Find and collect the TOTAL APPLICATIONS element
        try:
            total_applicants.append(driver.find_element(By.CSS_SELECTOR, '.jobs-unified-top-card__job-insight--highlight').text)
        except:
            total_applicants.append('0 applicant')
        
        ## Find and collect the LINK element
        try:
            link.append(job.find_element(By.CSS_SELECTOR, '.job-card-list__title').get_attribute('href')) 
        except:
            link.append('No link')
    print(f'''scraping page {i} successfully!''')
    i = i + 1        
            

scraping page 1 successfully!
scraping page 2 successfully!
scraping page 3 successfully!
scraping page 4 successfully!
scraping page 5 successfully!
scraping page 6 successfully!
scraping page 7 successfully!
scraping page 8 successfully!
scraping page 9 successfully!
scraping page 10 successfully!
scraping page 11 successfully!
scraping page 12 successfully!
scraping page 13 successfully!
scraping page 14 successfully!
scraping page 15 successfully!
scraping page 16 successfully!
scraping page 17 successfully!
scraping page 18 successfully!
scraping page 19 successfully!
scraping page 20 successfully!
scraping page 21 successfully!
scraping page 22 successfully!
scraping page 23 successfully!
scraping page 24 successfully!
scraping page 25 successfully!
scraping page 26 successfully!
scraping page 27 successfully!
scraping page 28 successfully!
scraping page 29 successfully!
scraping page 30 successfully!
scraping page 31 successfully!
scraping page 32 successfully!
scraping page 33 

In [180]:
#   TASK 4: DATA PREPROCESSING

##  Check the len of each list
print(len(date_posted))
print(len(title))
print(len(skill))
print(len(company))
print(len(job_type))
print(len(workplace_type))
print(len(company_size))
print(len(total_applicants))
print(len(link))

990
990
990
990
990
990
990
990
990


In [455]:
## Convert a dictionary of list into a dataframe and preprocess the data
job =   {
        'date_posted': date_posted,
        'title': title,
        'skill': skill,
        'company': company,
        'job_type': job_type,
        'workplace_type': workplace_type,
        'location': location,
        'company_size': company_size,
        'total_applicants': total_applicants,
        'job_link': link
        }

In [456]:
job_df1 = pd.DataFrame(job)
job_df1.head(3)

,date_posted,title,skill,company,job_type,workplace_type,location,company_size,total_applicants,job_link
0,1 week ago,Data Analyst,"Visualization, Data Analytics, Communication, ...",Revorec Recruitment Solutions Ltd,Full-time · Entry level,On-site,"Lelley, England, United Kingdom",11-50 employees · Staffing and Recruiting,See how you compare to 22 applicants. Try Prem...,https://www.linkedin.com/jobs/view/3613605144/...
1,1 week ago,RPG iSeries Senior Analyst,"RPG, IBM iSeries, Software Development Life Cy...",Vignetic,Full-time · Mid-Senior level,Remote,Sub-Saharan Africa,51-200 employees · Human Resources Services,See how you compare to 150 applicants. Try Pre...,https://www.linkedin.com/jobs/view/3584837094/...
2,1 week ago,Technical Lead,"C++, Software Development, English, Python (Pr...",Nimble Giant Entertainment,Full-time · Director,Remote,Latin America,201-500 employees · Computer Games,See how you compare to 34 applicants. Try Prem...,https://www.linkedin.com/jobs/view/3615790682/...


In [457]:
## Split company_size field into company_size and industry
job_df1[['company_size', 'industry']] = job_df1['company_size'].str.split(' · ', expand=True)

## Delect unneeded string and replace the blank space by 0 
job_df1['total_applicants'] = job_df1['total_applicants'].str.replace(r'\D', '', regex=True)        ## Just keep the number in the string
job_df1['total_applicants'] = job_df1['total_applicants'].apply(lambda x: 0 if x == '' else x)      ## Replace the space by 0

job_df1.head(3)


,date_posted,title,skill,company,job_type,workplace_type,location,company_size,total_applicants,job_link,industry
0,1 week ago,Data Analyst,"Visualization, Data Analytics, Communication, ...",Revorec Recruitment Solutions Ltd,Full-time · Entry level,On-site,"Lelley, England, United Kingdom",11-50 employees,22,https://www.linkedin.com/jobs/view/3613605144/...,Staffing and Recruiting
1,1 week ago,RPG iSeries Senior Analyst,"RPG, IBM iSeries, Software Development Life Cy...",Vignetic,Full-time · Mid-Senior level,Remote,Sub-Saharan Africa,51-200 employees,150,https://www.linkedin.com/jobs/view/3584837094/...,Human Resources Services
2,1 week ago,Technical Lead,"C++, Software Development, English, Python (Pr...",Nimble Giant Entertainment,Full-time · Director,Remote,Latin America,201-500 employees,34,https://www.linkedin.com/jobs/view/3615790682/...,Computer Games


In [458]:
## Add scraping date
job_df1['scraping_date'] = date.today()

In [459]:
## Add job id 
job_df1['job_id'] = list(range(1, len(job_df1)+1))  ## Range from 1 to the length of job_df1 (+1)

job_df1.head(3)

,date_posted,title,skill,company,job_type,workplace_type,location,company_size,total_applicants,job_link,industry,scraping_date,job_id
0,1 week ago,Data Analyst,"Visualization, Data Analytics, Communication, ...",Revorec Recruitment Solutions Ltd,Full-time · Entry level,On-site,"Lelley, England, United Kingdom",11-50 employees,22,https://www.linkedin.com/jobs/view/3613605144/...,Staffing and Recruiting,2023-06-08,1
1,1 week ago,RPG iSeries Senior Analyst,"RPG, IBM iSeries, Software Development Life Cy...",Vignetic,Full-time · Mid-Senior level,Remote,Sub-Saharan Africa,51-200 employees,150,https://www.linkedin.com/jobs/view/3584837094/...,Human Resources Services,2023-06-08,2
2,1 week ago,Technical Lead,"C++, Software Development, English, Python (Pr...",Nimble Giant Entertainment,Full-time · Director,Remote,Latin America,201-500 employees,34,https://www.linkedin.com/jobs/view/3615790682/...,Computer Games,2023-06-08,3


In [460]:
## TASK 4.1: Create and Transform job_list dataframe

In [461]:
job_df = job_df1.copy()

In [462]:
job_df = job_df.drop(columns=['skill'])

In [463]:
job_df.head(3)

,date_posted,title,company,job_type,workplace_type,location,company_size,total_applicants,job_link,industry,scraping_date,job_id
0,1 week ago,Data Analyst,Revorec Recruitment Solutions Ltd,Full-time · Entry level,On-site,"Lelley, England, United Kingdom",11-50 employees,22,https://www.linkedin.com/jobs/view/3613605144/...,Staffing and Recruiting,2023-06-08,1
1,1 week ago,RPG iSeries Senior Analyst,Vignetic,Full-time · Mid-Senior level,Remote,Sub-Saharan Africa,51-200 employees,150,https://www.linkedin.com/jobs/view/3584837094/...,Human Resources Services,2023-06-08,2
2,1 week ago,Technical Lead,Nimble Giant Entertainment,Full-time · Director,Remote,Latin America,201-500 employees,34,https://www.linkedin.com/jobs/view/3615790682/...,Computer Games,2023-06-08,3


In [464]:
job_df['job_type'].value_counts()

job_type
Full-time · Mid-Senior level                                                       521
Contract · Entry level                                                             117
Full-time · Entry level                                                             99
Full-time                                                                           75
Full-time · Associate                                                               44
                                                                                  ... 
$110,000/yr - $125,000/yr (from job description) · Full-time · Mid-Senior level      1
$48,000/yr - $62,500/yr (from job description) · Full-time · Entry level             1
$74,000/yr - $127,000/yr (from job description) · Full-time · Entry level            1
$28.84/hr (from job description) · Full-time · Mid-Senior level                      1
$68,000/yr - $118,000/yr · Full-time · Entry level                                   1
Name: count, Length: 76, dtype: in

In [465]:
y = list(map(lambda x: x.replace(x.split(' · ')[0], '') if(x.split(' · ')[0] not in ['Full-time', 'Contract', 'Internship', 'Part-time', 'Temporary', 'Unknown']) else x, job_df['job_type']))
    ##  Explain
    ### 1/ x.replace(x.split(' · ')[0], '') --> The function splits the string x based on " ." and returns the first element (ie [0])
    ### 2/ ... if(x.split(' · ')[0] not in ['Full-time', 'Contract', 'Internship', 'Part-time', 'Temporary', 'Unknown']) else x 
            #--> if the first element is not in the condition return ''
            #--> if the first element is in the condition then return x

In [466]:
job_df['test4'] = y

In [467]:
job_df['test4'].value_counts()

test4
Full-time · Mid-Senior level            521
Contract · Entry level                  117
Full-time · Entry level                  99
Full-time                                75
Full-time · Associate                    44
Contract · Mid-Senior level              36
 · Full-time · Mid-Senior level          23
 · Full-time                             18
Contract · Associate                     11
 · Full-time · Entry level               11
 · Contract · Entry level                 9
Full-time · Director                      7
Full-time · Executive                     4
Part-time · Mid-Senior level              3
Temporary · Mid-Senior level              2
Contract                                  2
 · Full-time · Associate                  1
Internship · Internship                   1
Full-time · Internship                    1
 · Information Technology & Services      1
 · Contract                               1
 · Internship · Mid-Senior level          1
 · Contract · Mid-Senior l

In [468]:
z = list(map(lambda x: ' '.join(x.split(' · ')).strip(), job_df['test4']))
##  Exlain
### x.split(' · '): split strings based on " . "
### ...strip(): remove terminal whitespace
### ' '.join(x.split(' · ')).strip(): concatenate elements with space ' '

In [469]:
job_df['job_type'] = z
job_df.head(5)

,date_posted,title,company,job_type,workplace_type,location,company_size,total_applicants,job_link,industry,scraping_date,job_id,test4
0,1 week ago,Data Analyst,Revorec Recruitment Solutions Ltd,Full-time Entry level,On-site,"Lelley, England, United Kingdom",11-50 employees,22,https://www.linkedin.com/jobs/view/3613605144/...,Staffing and Recruiting,2023-06-08,1,Full-time · Entry level
1,1 week ago,RPG iSeries Senior Analyst,Vignetic,Full-time Mid-Senior level,Remote,Sub-Saharan Africa,51-200 employees,150,https://www.linkedin.com/jobs/view/3584837094/...,Human Resources Services,2023-06-08,2,Full-time · Mid-Senior level
2,1 week ago,Technical Lead,Nimble Giant Entertainment,Full-time Director,Remote,Latin America,201-500 employees,34,https://www.linkedin.com/jobs/view/3615790682/...,Computer Games,2023-06-08,3,Full-time · Director
3,1 week ago,DataOps Engineer,Andela,Contract Mid-Senior level,Remote,Latin America,201-500 employees,73,https://www.linkedin.com/jobs/view/3613237663/...,IT Services and IT Consulting,2023-06-08,4,Contract · Mid-Senior level
4,2 days ago,Privacy Analyst,6sense,Contract Entry level,Remote,NAMER,"1,001-5,000 employees",619,https://www.linkedin.com/jobs/view/3578344324/...,Software Development,2023-06-08,5,· Contract · Entry level


In [470]:
job_df[['test4', 'job_level', 'test']] = job_df['test4'].str.split(' · ', expand=True)

job_df


,date_posted,title,company,job_type,workplace_type,location,company_size,total_applicants,job_link,industry,scraping_date,job_id,test4,job_level,test
0,1 week ago,Data Analyst,Revorec Recruitment Solutions Ltd,Full-time Entry level,On-site,"Lelley, England, United Kingdom",11-50 employees,22,https://www.linkedin.com/jobs/view/3613605144/...,Staffing and Recruiting,2023-06-08,1,Full-time,Entry level,None
1,1 week ago,RPG iSeries Senior Analyst,Vignetic,Full-time Mid-Senior level,Remote,Sub-Saharan Africa,51-200 employees,150,https://www.linkedin.com/jobs/view/3584837094/...,Human Resources Services,2023-06-08,2,Full-time,Mid-Senior level,None
2,1 week ago,Technical Lead,Nimble Giant Entertainment,Full-time Director,Remote,Latin America,201-500 employees,34,https://www.linkedin.com/jobs/view/3615790682/...,Computer Games,2023-06-08,3,Full-time,Director,None
3,1 week ago,DataOps Engineer,Andela,Contract Mid-Senior level,Remote,Latin America,201-500 employees,73,https://www.linkedin.com/jobs/view/3613237663/...,IT Services and IT Consulting,2023-06-08,4,Contract,Mid-Senior level,None
4,2 days ago,Privacy Analyst,6sense,Contract Entry level,Remote,NAMER,"1,001-5,000 employees",619,https://www.linkedin.com/jobs/view/3578344324/...,Software Development,2023-06-08,5,,Contract,Entry level
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,3 weeks ago,Business Analyst 3,"First Tek, Inc.",Contract Mid-Senior level,On-site,"Vancouver, WA","1,001-5,000 employees",0,https://www.linkedin.com/jobs/view/3623217544/...,IT Services and IT Consulting,2023-06-08,986,Contract,Mid-Senior level,None
986,3 weeks ago,Business Analyst,RAPS Consulting Inc,Contract Entry level,On-site,"Edison, NJ",51-200 employees,0,https://www.linkedin.com/jobs/view/3623204148/...,IT Services and IT Consulting,2023-06-08,987,Contract,Entry level,None
987,3 weeks ago,Business Analyst,Lynker,Full-time Entry level,Unknown,"Charleston, SC",201-500 employees,0,https://www.linkedin.com/jobs/view/3623233561/...,Environmental Services,2023-06-08,988,,Full-time,Entry level
988,1 week ago,Systems Analyst - Senior,J&M Group,Contract Mid-Senior level,On-site,"Mississauga, ON",11-50 employees,0,https://www.linkedin.com/jobs/view/3622593874/...,IT Services and IT Consulting,2023-06-08,989,Contract,Mid-Senior level,None


In [471]:
job_df.drop(columns=['job_type', 'test'], inplace=True)
job_df.rename(columns = {'test4':'job_type'}, inplace = True)
job_df.head()


,date_posted,title,company,workplace_type,location,company_size,total_applicants,job_link,industry,scraping_date,job_id,job_type,job_level
0,1 week ago,Data Analyst,Revorec Recruitment Solutions Ltd,On-site,"Lelley, England, United Kingdom",11-50 employees,22,https://www.linkedin.com/jobs/view/3613605144/...,Staffing and Recruiting,2023-06-08,1,Full-time,Entry level
1,1 week ago,RPG iSeries Senior Analyst,Vignetic,Remote,Sub-Saharan Africa,51-200 employees,150,https://www.linkedin.com/jobs/view/3584837094/...,Human Resources Services,2023-06-08,2,Full-time,Mid-Senior level
2,1 week ago,Technical Lead,Nimble Giant Entertainment,Remote,Latin America,201-500 employees,34,https://www.linkedin.com/jobs/view/3615790682/...,Computer Games,2023-06-08,3,Full-time,Director
3,1 week ago,DataOps Engineer,Andela,Remote,Latin America,201-500 employees,73,https://www.linkedin.com/jobs/view/3613237663/...,IT Services and IT Consulting,2023-06-08,4,Contract,Mid-Senior level
4,2 days ago,Privacy Analyst,6sense,Remote,NAMER,"1,001-5,000 employees",619,https://www.linkedin.com/jobs/view/3578344324/...,Software Development,2023-06-08,5,,Contract


In [472]:
## Re-arrange columnns
job_df = job_df.reindex(columns=[ 'job_id', 'date_posted', 'scraping_date', 'title', 'location', 'company', 'industry', 'job_level', 'job_type', 'workplace_type', 'company_size', 'total_applicants', 'job_link'])
job_df.drop(columns=['job_link'], inplace=True)
job_df.head()


,job_id,date_posted,scraping_date,title,location,company,industry,job_level,job_type,workplace_type,company_size,total_applicants
0,1,1 week ago,2023-06-08,Data Analyst,"Lelley, England, United Kingdom",Revorec Recruitment Solutions Ltd,Staffing and Recruiting,Entry level,Full-time,On-site,11-50 employees,22
1,2,1 week ago,2023-06-08,RPG iSeries Senior Analyst,Sub-Saharan Africa,Vignetic,Human Resources Services,Mid-Senior level,Full-time,Remote,51-200 employees,150
2,3,1 week ago,2023-06-08,Technical Lead,Latin America,Nimble Giant Entertainment,Computer Games,Director,Full-time,Remote,201-500 employees,34
3,4,1 week ago,2023-06-08,DataOps Engineer,Latin America,Andela,IT Services and IT Consulting,Mid-Senior level,Contract,Remote,201-500 employees,73
4,5,2 days ago,2023-06-08,Privacy Analyst,NAMER,6sense,Software Development,Contract,,Remote,"1,001-5,000 employees",619


In [473]:
job_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   job_id            990 non-null    int64 
 1   date_posted       990 non-null    object
 2   scraping_date     990 non-null    object
 3   title             990 non-null    object
 4   location          990 non-null    object
 5   company           990 non-null    object
 6   industry          974 non-null    object
 7   job_level         913 non-null    object
 8   job_type          990 non-null    object
 9   workplace_type    990 non-null    object
 10  company_size      990 non-null    object
 11  total_applicants  990 non-null    object
dtypes: int64(1), object(11)
memory usage: 92.9+ KB


In [474]:
job_df['scraping_date'] = pd.to_datetime(job_df['scraping_date'])
job_df['total_applicants'] = job_df['total_applicants'].astype(int)


In [475]:
job_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   job_id            990 non-null    int64         
 1   date_posted       990 non-null    object        
 2   scraping_date     990 non-null    datetime64[ns]
 3   title             990 non-null    object        
 4   location          990 non-null    object        
 5   company           990 non-null    object        
 6   industry          974 non-null    object        
 7   job_level         913 non-null    object        
 8   job_type          990 non-null    object        
 9   workplace_type    990 non-null    object        
 10  company_size      990 non-null    object        
 11  total_applicants  990 non-null    int64         
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 92.9+ KB


In [476]:
job_df.head()


,job_id,date_posted,scraping_date,title,location,company,industry,job_level,job_type,workplace_type,company_size,total_applicants
0,1,1 week ago,2023-06-08,Data Analyst,"Lelley, England, United Kingdom",Revorec Recruitment Solutions Ltd,Staffing and Recruiting,Entry level,Full-time,On-site,11-50 employees,22
1,2,1 week ago,2023-06-08,RPG iSeries Senior Analyst,Sub-Saharan Africa,Vignetic,Human Resources Services,Mid-Senior level,Full-time,Remote,51-200 employees,150
2,3,1 week ago,2023-06-08,Technical Lead,Latin America,Nimble Giant Entertainment,Computer Games,Director,Full-time,Remote,201-500 employees,34
3,4,1 week ago,2023-06-08,DataOps Engineer,Latin America,Andela,IT Services and IT Consulting,Mid-Senior level,Contract,Remote,201-500 employees,73
4,5,2 days ago,2023-06-08,Privacy Analyst,NAMER,6sense,Software Development,Contract,,Remote,"1,001-5,000 employees",619


In [477]:
## TASK 4.2: Create and Transform Job_skill dataframe
job_skill = job_df1[['job_id', 'skill']].copy()


In [478]:
job_skill.head()

,job_id,skill
0,1,"Visualization, Data Analytics, Communication, ..."
1,2,"RPG, IBM iSeries, Software Development Life Cy..."
2,3,"C++, Software Development, English, Python (Pr..."
3,4,"Scala, Apache Kafka, Terraform, Java, Apache S..."
4,5,"Privacy Regulations, Privacy Law, Data Privacy..."


In [479]:
job_skill[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']] = job_skill['skill'].str.split(', ', expand=True)

In [480]:
job_skill.tail()

,job_id,skill,1,2,3,4,5,6,7,8,9,10,11,12
985,986,"Business Analysis, Requirements Gathering, BPA...",Business Analysis,Requirements Gathering,BPA,Analytical Skills,Diagram Design,Integrated Production,Observation,Communication,Data Analysis,and Business Requirements,None,None
986,987,"Technical Reviews, Business Case Preparation, ...",Technical Reviews,Business Case Preparation,Non-Functional Requirements,Requirements Traceability,Business Analysis,Coordinating Activities,Key Performance Indicators,Business Rules,Business Requirements,and Process Modeling,None,None
987,988,"Requirement Specifications, User Acceptance Te...",Requirement Specifications,User Acceptance Testing,Acceptance Testing,Technical Requirements,Business Analysis,User Requirements,Communication,Quality Assurance,System Development,and Systems Analysis,None,None
988,989,"Microsoft Dynamics, Microsoft Dynamics AX, Fin...",Microsoft Dynamics,Microsoft Dynamics AX,Finance,Dynamics,Systems Analysis,Databases,Microsoft Dynamics 365,Analytical Skills,Business Analysis,and Requirements Gathering,None,None
989,990,"Java, REST APIs, RabbitMQ, Apache Kafka, Angul...",Java,REST APIs,RabbitMQ,Apache Kafka,Angular,Service-Oriented Architecture (SOA),Spring Boot,and Microservices,None,None,None,None


In [481]:
## Unpivot job_skill
job_skill_unpivot = pd.melt(job_skill, id_vars='job_id', value_vars=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'])
##  Explain
### melt() làm tan chảy 12 cột thành một cột duy nhất

In [482]:
job_skill_unpivot = job_skill_unpivot.loc[job_skill_unpivot['value'].notna()]

In [483]:
job_skill_unpivot.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9293 entries, 0 to 11872
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   job_id    9293 non-null   int64 
 1   variable  9293 non-null   object
 2   value     9293 non-null   object
dtypes: int64(1), object(2)
memory usage: 290.4+ KB


In [484]:
x = list(map(lambda x: x.replace('and', '') if(x.split(' ')[0] == 'and') else x, job_skill_unpivot['value']))

In [485]:
job_skill_unpivot['value_after_remove_and_lambda']  = x

In [486]:
job_skill_unpivot

,job_id,variable,value,value_after_remove_and_lambda
0,1,1,Visualization,Visualization
1,2,1,RPG,RPG
2,3,1,C++,C++
3,4,1,Scala,Scala
4,5,1,Privacy Regulations,Privacy Regulations
...,...,...,...,...
11820,931,12,Load (ETL),Load (ETL)
11822,933,12,and Databases,Databases
11829,940,12,and Data Quality,Data Quality
11853,964,12,and Business Analysis,Business Analysis


In [487]:
job_skill_unpivot = job_skill_unpivot.drop(columns=['variable', 'value'])

In [488]:
job_skill_unpivot

,job_id,value_after_remove_and_lambda
0,1,Visualization
1,2,RPG
2,3,C++
3,4,Scala
4,5,Privacy Regulations
...,...,...
11820,931,Load (ETL)
11822,933,Databases
11829,940,Data Quality
11853,964,Business Analysis


In [489]:
job_skill_unpivot.rename(columns={'value_after_remove_and_lambda': 'skill'}, inplace=True)


In [490]:
job_skill_unpivot.head()


,job_id,skill
0,1,Visualization
1,2,RPG
2,3,C++
3,4,Scala
4,5,Privacy Regulations


In [491]:
# TASK 5: EXPORT job_list AND job_skill INTO EXCEL FILE

In [492]:
writer = pd.ExcelWriter('/Users/namhuynh/Desktop/WS(linkedinJOBs)/job_list_final.xlsx')
job_df.to_excel(writer)

workbook = writer.book
worksheet = writer.sheets["Sheet1"]

writer.close()

In [493]:
writer = pd.ExcelWriter('/Users/namhuynh/Desktop/WS(linkedinJOBs)/job_skill_final.xlsx')
job_skill_unpivot.to_excel(writer)

workbook = writer.book
worksheet = writer.sheets["Sheet1"]

writer.close()

In [496]:
## Export into CSV file
job_df.to_csv(r'/Users/namhuynh/Desktop/WS(linkedinJOBs)/job_df.csv')
job_skill_unpivot.to_csv(r'/Users/namhuynh/Desktop/WS(linkedinJOBs)/job_skill.csv')